In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from google.colab import drive
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the CSV file with robust handling
try:
    df = pd.read_csv(
        '/content/drive/My Drive/output.csv',  # Path to the file
        on_bad_lines='skip',  # Skip problematic rows
        escapechar="\\",      # Handle escape characters
        encoding='utf-8',     # Ensure UTF-8 encoding
        engine='python'       # Use Python engine for better error handling
    )
    print("CSV file loaded successfully!")
except Exception as e:
    print(f"Error loading CSV file: {e}")

# Display the first few rows to verify
# df.head()

CSV file loaded successfully!


In [ ]:
df.head()

,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,politics,Politico,0,https://www.politico.com/news/magazine/2019/11...,Can the Impeachment Hearings Actually Change A...,2019-11-13,NaN,"This happens for different reasons , but a key...","This happens for different reasons, but a key ...",www.politico.com,left,bpp2Cqpvyi2ER5Lr
1,china,Bloomberg,1,https://www.bloomberg.com/news/articles/2018-0...,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,"Brendan Scott, Enda Curran",LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,www.bloomberg.com,center,PnVV1v7F0D8bhd5X
2,terrorism,Washington Times,2,http://www.washingtontimes.com/news/2014/sep/1...,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,Stephen Dinan,The Pentagon ’ s top general said Tuesday that...,The Pentagon’s top general said Tuesday that t...,www.washingtontimes.com,right,pZlutCuZNm832qqK
3,politics,Salon,0,http://www.salon.com/2014/11/12/americas_post_...,America’s post-midterm inferno: Tea Party garb...,2014-11-12,Robert Hennelly,A full week after the alleged “ shellacking “ ...,A full week after the alleged “shellacking “ o...,www.salon.com,left,aMGkvFlpLyQrX6iu
4,politics,CNN (Web News),0,http://www.cnn.com/2014/01/08/politics/christi...,E-mails among Christie appointees suggest poli...,2014-01-08,Catherine E. Shoichet,Story highlights Gergen : `` It feeds into thi...,"Story highlights Gergen: ""It feeds into this n...",www.cnn.com,left,Ludq48gCzDiKtnmX


In [ ]:

df = df.drop(columns=['source_url', 'url','content_original','authors'])

In [ ]:
df

,topic,source,bias,title,date,content,bias_text,ID
0,politics,Politico,0,Can the Impeachment Hearings Actually Change A...,2019-11-13,"This happens for different reasons , but a key...",left,bpp2Cqpvyi2ER5Lr
1,china,Bloomberg,1,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,center,PnVV1v7F0D8bhd5X
2,terrorism,Washington Times,2,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,The Pentagon ’ s top general said Tuesday that...,right,pZlutCuZNm832qqK
3,politics,Salon,0,America’s post-midterm inferno: Tea Party garb...,2014-11-12,A full week after the alleged “ shellacking “ ...,left,aMGkvFlpLyQrX6iu
4,politics,CNN (Web News),0,E-mails among Christie appointees suggest poli...,2014-01-08,Story highlights Gergen : `` It feeds into thi...,left,Ludq48gCzDiKtnmX
...,...,...,...,...,...,...,...,...
37549,media_bias,Fox News,2,Summit meeting: Can there be a 'reset' between...,2016-11-22,If Donald Trump can sit down with Mitt Romney ...,right,aDXaucBITOvKX6Sl
37550,education,Vox,0,All of West Virginia’s teachers have been on s...,2018-03-03,Thousands of public school teachers across Wes...,left,78Akx2nSokxbCMd0
37551,elections,The Hill,1,Kirsten Gillibrand officially announces White ...,2019-03-17,Sen. Kirsten Gillibrand Kirsten GillibrandTo w...,center,vvAgpKy0PL1h8990
37552,education,Reason,2,"Why We Need School Choice: ""Obama Administrati...",2017-01-23,"Welcome to National School Choice Week , an an...",right,aWlri1iRio9Q6Rfe


In [ ]:
# Combine 'title' and 'content' as input text
df['text'] = df['title'] + " " + df['content']

In [ ]:
df

,topic,source,bias,title,date,content,bias_text,ID,text
0,politics,Politico,0,Can the Impeachment Hearings Actually Change A...,2019-11-13,"This happens for different reasons , but a key...",left,bpp2Cqpvyi2ER5Lr,Can the Impeachment Hearings Actually Change A...
1,china,Bloomberg,1,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,center,PnVV1v7F0D8bhd5X,Trump’s Tariff Barrage Pushes China Fight to P...
2,terrorism,Washington Times,2,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,The Pentagon ’ s top general said Tuesday that...,right,pZlutCuZNm832qqK,Top general: Failed Foley rescue was ‘highest-...
3,politics,Salon,0,America’s post-midterm inferno: Tea Party garb...,2014-11-12,A full week after the alleged “ shellacking “ ...,left,aMGkvFlpLyQrX6iu,America’s post-midterm inferno: Tea Party garb...
4,politics,CNN (Web News),0,E-mails among Christie appointees suggest poli...,2014-01-08,Story highlights Gergen : `` It feeds into thi...,left,Ludq48gCzDiKtnmX,E-mails among Christie appointees suggest poli...
...,...,...,...,...,...,...,...,...,...
37549,media_bias,Fox News,2,Summit meeting: Can there be a 'reset' between...,2016-11-22,If Donald Trump can sit down with Mitt Romney ...,right,aDXaucBITOvKX6Sl,Summit meeting: Can there be a 'reset' between...
37550,education,Vox,0,All of West Virginia’s teachers have been on s...,2018-03-03,Thousands of public school teachers across Wes...,left,78Akx2nSokxbCMd0,All of West Virginia’s teachers have been on s...
37551,elections,The Hill,1,Kirsten Gillibrand officially announces White ...,2019-03-17,Sen. Kirsten Gillibrand Kirsten GillibrandTo w...,center,vvAgpKy0PL1h8990,Kirsten Gillibrand officially announces White ...
37552,education,Reason,2,"Why We Need School Choice: ""Obama Administrati...",2017-01-23,"Welcome to National School Choice Week , an an...",right,aWlri1iRio9Q6Rfe,"Why We Need School Choice: ""Obama Administrati..."


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# Split data into features and labels
X = df['text']  # Assuming your text column is named 'text'
y = df['bias']  # Assuming your labels column is named 'label'

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# TF-IDF vectorization
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)  # Unigrams and bigrams, limit features to 5000
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# Evaluate on test set
y_pred = nb.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))




Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.60      0.59      2601
           1       0.59      0.51      0.55      2163
           2       0.60      0.65      0.63      2747

    accuracy                           0.59      7511
   macro avg       0.59      0.59      0.59      7511
weighted avg       0.59      0.59      0.59      7511

Confusion Matrix:
[[1551  371  679]
 [ 562 1105  496]
 [ 562  387 1798]]


In [ ]:
# Save TF-IDF vectorizer and model for reuse
import joblib
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(nb, 'naive_bayes_model.pkl')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Split data into features and labels
X = df['text']  # Assuming your text column is named 'text'
y = df['bias']  # Assuming your labels column is named 'label'

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# TF-IDF vectorization
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)  # Unigrams and bigrams, limit features to 5000
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# Evaluate on test set
y_pred = nb.predict(X_test_tfidf)

# Overall metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # 'weighted' handles class imbalance
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Overall Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)



Overall Metrics:
Accuracy: 0.59
Precision: 0.59
Recall: 0.59
F1-Score: 0.59

Confusion Matrix:
[[1551  371  679]
 [ 562 1105  496]
 [ 562  387 1798]]


In [ ]:

# Save TF-IDF vectorizer and model for reuse
import joblib
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(nb, 'naive_bayes_model.pkl')